# Advanced Fluid Dynamics Theory in Financial Markets

## Table of Contents
1. [Mathematical Foundations](#mathematical-foundations)
2. [Navier-Stokes Equations for Market Flow](#navier-stokes-equations)
3. [Vorticity and Circulation Analysis](#vorticity-analysis)
4. [Reynolds Number in Market Regimes](#reynolds-number)
5. [Energy Cascade Theory](#energy-cascade)
6. [Boundary Layer Dynamics](#boundary-layer)
7. [Computational Methods](#computational-methods)
8. [Advanced Pattern Recognition](#advanced-patterns)

## Introduction

This notebook explores the deep mathematical foundations of applying computational fluid dynamics (CFD) to financial market analysis. We develop rigorous mathematical models that map market microstructure to fluid dynamics principles, enabling sophisticated pattern recognition and regime detection.

### Core Principle: Market as Continuous Medium

In our framework, we treat the market as a continuous medium where:
- **Order flow** becomes fluid flow velocity field `v(x,t)`
- **Price levels** become spatial coordinates `x`
- **Market depth** becomes fluid density `ρ(x,t)`
- **Bid-ask spread** becomes pressure field `p(x,t)`
- **Trading volume** becomes fluid flux `φ(x,t)`

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import ndimage
from scipy.ndimage import gaussian_filter
from scipy.signal import find_peaks
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from matplotlib.patches import Circle, Rectangle
from matplotlib.collections import LineCollection
import warnings
warnings.filterwarnings('ignore')

# Set style for better visualizations
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

## Mathematical Foundations

### 1. Field Theory for Market Data

We define scalar and vector fields to represent market properties:

#### Scalar Fields:
- **Price field**: $P(x,t)$ where $x$ represents price level, $t$ is time
- **Density field**: $\rho(x,t) = \text{market depth at price level } x$
- **Pressure field**: $p(x,t) = \text{bid-ask pressure differential}$
- **Temperature field**: $T(x,t) = \text{volatility measure}$

#### Vector Fields:
- **Velocity field**: $\vec{v}(x,t) = (v_x(x,t), v_t(x,t))$
- **Force field**: $\vec{F}(x,t) = \text{net buying/selling pressure}$

### 2. Continuity Equation

Conservation of market liquidity:
$$\frac{\partial \rho}{\partial t} + \nabla \cdot (\rho \vec{v}) = S$$

Where $S$ represents external liquidity sources/sinks (market makers, institutional flows).

In [ ]:
class AdvancedFluidMarketModel:
    """
    Advanced fluid dynamics model for financial markets with rigorous mathematical foundations.
    """
    
    def __init__(self, grid_size=100, time_steps=1000):
        self.grid_size = grid_size
        self.time_steps = time_steps
        self.dx = 1.0 / grid_size  # Spatial resolution
        self.dt = 0.001  # Time step
        
        # Initialize fields
        self.density = np.ones((grid_size, grid_size))
        self.velocity_x = np.zeros((grid_size, grid_size))
        self.velocity_y = np.zeros((grid_size, grid_size))
        self.pressure = np.zeros((grid_size, grid_size))
        self.vorticity = np.zeros((grid_size, grid_size))
        
        # Physical parameters
        self.viscosity = 0.01  # Market friction
        self.reynolds_number = 1000  # Regime indicator
        
    def compute_gradient(self, field):
        """Compute spatial gradient using central differences"""
        grad_x = np.gradient(field, axis=1) / self.dx
        grad_y = np.gradient(field, axis=0) / self.dx
        return grad_x, grad_y
    
    def compute_divergence(self, vx, vy):
        """Compute divergence of velocity field"""
        dvx_dx = np.gradient(vx, axis=1) / self.dx
        dvy_dy = np.gradient(vy, axis=0) / self.dx
        return dvx_dx + dvy_dy
    
    def compute_vorticity(self, vx, vy):
        """Compute vorticity (curl of velocity field)"""
        dvx_dy = np.gradient(vx, axis=0) / self.dx
        dvy_dx = np.gradient(vy, axis=1) / self.dx
        return dvy_dx - dvx_dy
    
    def compute_laplacian(self, field):
        """Compute Laplacian using finite differences"""
        laplacian = (np.roll(field, 1, axis=0) + np.roll(field, -1, axis=0) +
                    np.roll(field, 1, axis=1) + np.roll(field, -1, axis=1) -
                    4 * field) / (self.dx**2)
        return laplacian

# Initialize model
model = AdvancedFluidMarketModel()
print("Advanced Fluid Market Model initialized")
print(f"Grid size: {model.grid_size}x{model.grid_size}")
print(f"Spatial resolution: {model.dx:.4f}")
print(f"Time step: {model.dt}")

## Navier-Stokes Equations for Market Flow

### Momentum Conservation

The core equation governing market flow dynamics:

$$\frac{\partial \vec{v}}{\partial t} + (\vec{v} \cdot \nabla)\vec{v} = -\frac{1}{\rho}\nabla p + \nu \nabla^2 \vec{v} + \vec{f}$$

Where:
- $\vec{v}$ = velocity field (order flow direction and magnitude)
- $p$ = pressure field (market pressure from bid-ask dynamics)
- $\rho$ = density field (market depth/liquidity)
- $\nu$ = kinematic viscosity (market friction/transaction costs)
- $\vec{f}$ = external force field (news, institutional flows)

### Market Interpretation:

1. **Advection term** $(\vec{v} \cdot \nabla)\vec{v}$: Non-linear momentum transfer
2. **Pressure gradient** $-\frac{1}{\rho}\nabla p$: Force from price imbalances
3. **Viscous term** $\nu \nabla^2 \vec{v}$: Diffusive effects from market friction
4. **External forces** $\vec{f}$: Impact of news, large orders, etc.

In [ ]:
def solve_navier_stokes_step(model, external_force_x=None, external_force_y=None):
    """
    Single time step of Navier-Stokes equation solution using finite differences.
    """
    vx, vy = model.velocity_x, model.velocity_y
    rho = model.density
    p = model.pressure
    dt, dx = model.dt, model.dx
    nu = model.viscosity
    
    # Compute gradients
    dp_dx, dp_dy = model.compute_gradient(p)
    
    # Compute Laplacians for viscous terms
    lap_vx = model.compute_laplacian(vx)
    lap_vy = model.compute_laplacian(vy)
    
    # Advection terms (non-linear)
    dvx_dx, dvx_dy = model.compute_gradient(vx)
    dvy_dx, dvy_dy = model.compute_gradient(vy)
    
    advection_x = vx * dvx_dx + vy * dvx_dy
    advection_y = vx * dvy_dx + vy * dvy_dy
    
    # External forces (default to zero)
    if external_force_x is None:
        external_force_x = np.zeros_like(vx)
    if external_force_y is None:
        external_force_y = np.zeros_like(vy)
    
    # Update velocity using Navier-Stokes
    new_vx = vx + dt * (-advection_x - dp_dx/rho + nu * lap_vx + external_force_x)
    new_vy = vy + dt * (-advection_y - dp_dy/rho + nu * lap_vy + external_force_y)
    
    return new_vx, new_vy

def create_market_shock(model, center_x=50, center_y=50, intensity=5.0, radius=10):
    """
    Create a localized market shock (e.g., news event, large order)
    """
    x = np.arange(model.grid_size)
    y = np.arange(model.grid_size)
    X, Y = np.meshgrid(x, y)
    
    distance = np.sqrt((X - center_x)**2 + (Y - center_y)**2)
    shock = intensity * np.exp(-distance**2 / (2 * radius**2))
    
    return shock

# Demonstrate market shock propagation
shock_force_x = create_market_shock(model, intensity=2.0)
shock_force_y = create_market_shock(model, center_x=60, center_y=40, intensity=1.5)

print("Market shock created")
print(f"Maximum shock intensity: {np.max(shock_force_x):.2f}")

## Vorticity and Circulation Analysis

### Vorticity Definition

Vorticity measures the local rotation of the flow field:
$$\omega = \nabla \times \vec{v} = \frac{\partial v_y}{\partial x} - \frac{\partial v_x}{\partial y}$$

### Market Interpretation:
- **Positive vorticity**: Counterclockwise rotation → Potential bullish reversal
- **Negative vorticity**: Clockwise rotation → Potential bearish reversal
- **High vorticity magnitude**: Strong rotational patterns → Trend change zones

### Circulation

Circulation around a closed contour:
$$\Gamma = \oint_C \vec{v} \cdot d\vec{l}$$

By Stokes' theorem: $\Gamma = \iint_S \omega \, dA$

In [ ]:
def analyze_vorticity_patterns(model):
    """
    Comprehensive vorticity analysis for pattern recognition
    """
    # Compute vorticity field
    omega = model.compute_vorticity(model.velocity_x, model.velocity_y)
    
    # Identify vortex cores (local extrema in vorticity)
    from scipy.ndimage import maximum_filter, minimum_filter
    
    # Find local maxima (positive vortices)
    local_max = (omega == maximum_filter(omega, size=5))
    positive_vortices = omega * local_max
    positive_vortices[positive_vortices < 0.1 * np.max(omega)] = 0
    
    # Find local minima (negative vortices)
    local_min = (omega == minimum_filter(omega, size=5))
    negative_vortices = omega * local_min
    negative_vortices[negative_vortices > 0.1 * np.min(omega)] = 0
    
    # Compute circulation around detected vortices
    circulation_strength = np.abs(positive_vortices) + np.abs(negative_vortices)
    
    return {
        'vorticity': omega,
        'positive_vortices': positive_vortices,
        'negative_vortices': negative_vortices,
        'circulation_strength': circulation_strength,
        'total_circulation': np.sum(circulation_strength)
    }

def detect_vortex_strength(vorticity_field, threshold=0.5):
    """
    Classify vortex strength based on Q-criterion and vorticity magnitude
    """
    # Compute Q-criterion (second invariant of velocity gradient tensor)
    dvx_dx = np.gradient(model.velocity_x, axis=1)
    dvx_dy = np.gradient(model.velocity_x, axis=0)
    dvy_dx = np.gradient(model.velocity_y, axis=1)
    dvy_dy = np.gradient(model.velocity_y, axis=0)
    
    # Q = 0.5 * (||Ω||² - ||S||²) where Ω is rotation, S is strain
    rotation_mag = 0.5 * (dvy_dx - dvx_dy)**2
    strain_mag = 0.5 * ((dvx_dx - dvy_dy)**2 + (dvy_dx + dvx_dy)**2)
    Q_criterion = 0.5 * (rotation_mag - strain_mag)
    
    # Classify vortex strength
    vortex_strength = np.abs(vorticity_field)
    
    weak_vortices = (vortex_strength > 0.1 * threshold) & (vortex_strength <= 0.5 * threshold)
    moderate_vortices = (vortex_strength > 0.5 * threshold) & (vortex_strength <= threshold)
    strong_vortices = vortex_strength > threshold
    
    return {
        'Q_criterion': Q_criterion,
        'weak_vortices': weak_vortices,
        'moderate_vortices': moderate_vortices,
        'strong_vortices': strong_vortices
    }

# Initialize synthetic velocity field for demonstration
x = np.linspace(0, 2*np.pi, model.grid_size)
y = np.linspace(0, 2*np.pi, model.grid_size)
X, Y = np.meshgrid(x, y)

# Create synthetic vortex pattern
model.velocity_x = np.sin(Y) * np.cos(X/2) + 0.3 * np.random.randn(*X.shape)
model.velocity_y = -np.sin(X) * np.cos(Y/2) + 0.3 * np.random.randn(*Y.shape)

# Analyze vorticity patterns
vorticity_analysis = analyze_vorticity_patterns(model)
vortex_classification = detect_vortex_strength(vorticity_analysis['vorticity'])

print("Vorticity Analysis Complete:")
print(f"Total circulation: {vorticity_analysis['total_circulation']:.3f}")
print(f"Strong vortices detected: {np.sum(vortex_classification['strong_vortices'])}")
print(f"Moderate vortices detected: {np.sum(vortex_classification['moderate_vortices'])}")

## Reynolds Number in Market Regimes

### Definition

Reynolds number characterizes the ratio of inertial to viscous forces:
$$Re = \frac{\rho V L}{\mu} = \frac{V L}{\nu}$$

Where:
- $V$ = characteristic velocity (order flow speed)
- $L$ = characteristic length (price range)
- $\nu$ = kinematic viscosity (market friction)

### Market Regime Classification:

1. **Low Reynolds (Re < 100)**: Laminar flow → Trending markets
2. **Moderate Reynolds (100 < Re < 1000)**: Transitional → Consolidation phases
3. **High Reynolds (Re > 1000)**: Turbulent flow → Choppy, volatile markets

### Critical Reynolds Number

The transition point where markets shift from trending to choppy behavior.

In [ ]:
class MarketReynoldsAnalyzer:
    """
    Analyze market regimes using Reynolds number and turbulence metrics
    """
    
    def __init__(self, model):
        self.model = model
        self.regime_history = []
        
    def compute_reynolds_number(self, velocity_field, characteristic_length=1.0):
        """
        Compute local Reynolds number across the market grid
        """
        # Characteristic velocity magnitude
        velocity_magnitude = np.sqrt(self.model.velocity_x**2 + self.model.velocity_y**2)
        
        # Avoid division by zero
        reynolds_number = (velocity_magnitude * characteristic_length) / \
                         (self.model.viscosity + 1e-8)
        
        return reynolds_number
    
    def classify_regime(self, reynolds_field):
        """
        Classify market regime based on Reynolds number distribution
        """
        mean_reynolds = np.mean(reynolds_field)
        std_reynolds = np.std(reynolds_field)
        
        # Regime thresholds
        laminar_threshold = 100
        turbulent_threshold = 1000
        
        # Classification logic
        if mean_reynolds < laminar_threshold:
            regime = 'laminar'  # Trending market
            confidence = 1.0 - (mean_reynolds / laminar_threshold)
        elif mean_reynolds > turbulent_threshold:
            regime = 'turbulent'  # Choppy market
            confidence = min(1.0, (mean_reynolds - turbulent_threshold) / turbulent_threshold)
        else:
            regime = 'transitional'  # Mixed regime
            confidence = 1.0 - abs(mean_reynolds - 550) / 450  # Distance from middle
        
        # Consider Reynolds number variability
        uniformity = 1.0 / (1.0 + std_reynolds / (mean_reynolds + 1e-8))
        
        return {
            'regime': regime,
            'confidence': max(0.0, min(1.0, confidence)),
            'mean_reynolds': mean_reynolds,
            'std_reynolds': std_reynolds,
            'uniformity': uniformity
        }
    
    def compute_turbulence_intensity(self):
        """
        Compute turbulence intensity as fluctuation measure
        """
        # Velocity fluctuations
        mean_vx = np.mean(self.model.velocity_x)
        mean_vy = np.mean(self.model.velocity_y)
        
        fluctuation_vx = self.model.velocity_x - mean_vx
        fluctuation_vy = self.model.velocity_y - mean_vy
        
        # Turbulent kinetic energy
        tke = 0.5 * (fluctuation_vx**2 + fluctuation_vy**2)
        mean_tke = np.mean(tke)
        
        # Mean velocity magnitude
        mean_velocity = np.sqrt(mean_vx**2 + mean_vy**2)
        
        # Turbulence intensity
        turbulence_intensity = np.sqrt(2 * mean_tke / 3) / (mean_velocity + 1e-8)
        
        return {
            'turbulent_kinetic_energy': tke,
            'mean_tke': mean_tke,
            'turbulence_intensity': turbulence_intensity
        }
    
    def analyze_current_regime(self):
        """
        Complete regime analysis for current time step
        """
        reynolds_field = self.compute_reynolds_number(None)
        regime_info = self.classify_regime(reynolds_field)
        turbulence_info = self.compute_turbulence_intensity()
        
        analysis = {
            **regime_info,
            **turbulence_info,
            'reynolds_field': reynolds_field
        }
        
        self.regime_history.append(analysis)
        return analysis

# Initialize Reynolds analyzer
reynolds_analyzer = MarketReynoldsAnalyzer(model)

# Analyze current regime
regime_analysis = reynolds_analyzer.analyze_current_regime()

print("Market Regime Analysis:")
print(f"Regime: {regime_analysis['regime']}")
print(f"Confidence: {regime_analysis['confidence']:.3f}")
print(f"Mean Reynolds number: {regime_analysis['mean_reynolds']:.2f}")
print(f"Turbulence intensity: {regime_analysis['turbulence_intensity']:.4f}")
print(f"Regime uniformity: {regime_analysis['uniformity']:.3f}")

## Energy Cascade Theory

### Kolmogorov's Theory Applied to Markets

Energy cascade in financial markets follows similar principles to turbulent flows:

1. **Large-scale injection**: Institutional orders, macroeconomic news
2. **Inertial range**: Energy transfer across different time scales
3. **Dissipation range**: Energy absorbed by market friction

### Energy Spectrum

Power spectral density follows Kolmogorov's -5/3 law:
$$E(k) = C \varepsilon^{2/3} k^{-5/3}$$

Where:
- $E(k)$ = energy spectrum
- $k$ = wavenumber (frequency)
- $\varepsilon$ = energy dissipation rate
- $C$ = Kolmogorov constant

In [ ]:
class EnergySpectrumAnalyzer:
    """
    Analyze energy spectrum and cascade in market data using Kolmogorov theory
    """
    
    def __init__(self, model):
        self.model = model
        
    def compute_kinetic_energy_spectrum(self, velocity_x, velocity_y):
        """
        Compute 2D kinetic energy spectrum
        """
        # Fourier transform of velocity components
        vx_fft = np.fft.fft2(velocity_x)
        vy_fft = np.fft.fft2(velocity_y)
        
        # Kinetic energy in Fourier space
        energy_fft = 0.5 * (np.abs(vx_fft)**2 + np.abs(vy_fft)**2)
        
        # Compute radial average
        kx = np.fft.fftfreq(self.model.grid_size, d=self.model.dx)
        ky = np.fft.fftfreq(self.model.grid_size, d=self.model.dx)
        KX, KY = np.meshgrid(kx, ky)
        K = np.sqrt(KX**2 + KY**2)
        
        # Define wavenumber bins
        k_bins = np.logspace(np.log10(1), np.log10(self.model.grid_size//2), 20)
        energy_spectrum = np.zeros(len(k_bins)-1)
        k_centers = np.zeros(len(k_bins)-1)
        
        for i in range(len(k_bins)-1):
            mask = (K >= k_bins[i]) & (K < k_bins[i+1])
            energy_spectrum[i] = np.sum(energy_fft[mask])
            k_centers[i] = np.sqrt(k_bins[i] * k_bins[i+1])
        
        return k_centers, energy_spectrum
    
    def fit_kolmogorov_spectrum(self, k, E_k):
        """
        Fit Kolmogorov -5/3 power law to energy spectrum
        """
        # Remove zeros and take log
        valid_idx = (E_k > 0) & (k > 0)
        log_k = np.log10(k[valid_idx])
        log_E = np.log10(E_k[valid_idx])
        
        if len(log_k) < 3:
            return None, None, 0.0
        
        # Linear regression for power law: log(E) = log(C) - (5/3)*log(k)
        A = np.vstack([log_k, np.ones(len(log_k))]).T
        slope, intercept = np.linalg.lstsq(A, log_E, rcond=None)[0]
        
        # Goodness of fit
        predicted_log_E = slope * log_k + intercept
        r_squared = 1 - np.sum((log_E - predicted_log_E)**2) / np.sum((log_E - np.mean(log_E))**2)
        
        # Theoretical Kolmogorov slope is -5/3 ≈ -1.667
        kolmogorov_deviation = abs(slope + 5/3)
        
        return slope, 10**intercept, r_squared, kolmogorov_deviation
    
    def compute_energy_dissipation_rate(self):
        """
        Estimate energy dissipation rate ε
        """
        # Compute velocity gradients
        dvx_dx, dvx_dy = self.model.compute_gradient(self.model.velocity_x)
        dvy_dx, dvy_dy = self.model.compute_gradient(self.model.velocity_y)
        
        # Strain rate tensor components
        S11 = dvx_dx
        S12 = 0.5 * (dvx_dy + dvy_dx)
        S22 = dvy_dy
        
        # Dissipation rate: ε = 2ν * S_ij * S_ij
        strain_rate_magnitude = S11**2 + 2*S12**2 + S22**2
        dissipation_rate = 2 * self.model.viscosity * strain_rate_magnitude
        
        return np.mean(dissipation_rate)
    
    def analyze_energy_cascade(self):
        """
        Complete energy cascade analysis
        """
        # Compute energy spectrum
        k, E_k = self.compute_kinetic_energy_spectrum(self.model.velocity_x, self.model.velocity_y)
        
        # Fit Kolmogorov spectrum
        slope, amplitude, r_squared, kolmogorov_dev = self.fit_kolmogorov_spectrum(k, E_k)
        
        # Compute dissipation rate
        dissipation_rate = self.compute_energy_dissipation_rate()
        
        # Integral length scale (large-scale characteristic)
        total_energy = np.sum(E_k)
        integral_scale = np.sum(E_k / k) / total_energy if total_energy > 0 else 0
        
        return {
            'wavenumbers': k,
            'energy_spectrum': E_k,
            'slope': slope,
            'amplitude': amplitude,
            'r_squared': r_squared,
            'kolmogorov_deviation': kolmogorov_dev,
            'dissipation_rate': dissipation_rate,
            'integral_scale': integral_scale
        }

# Initialize energy spectrum analyzer
energy_analyzer = EnergySpectrumAnalyzer(model)

# Perform energy cascade analysis
cascade_analysis = energy_analyzer.analyze_energy_cascade()

print("Energy Cascade Analysis:")
if cascade_analysis['slope'] is not None:
    print(f"Power law slope: {cascade_analysis['slope']:.3f}")
    print(f"Kolmogorov deviation: {cascade_analysis['kolmogorov_deviation']:.3f}")
    print(f"R-squared fit: {cascade_analysis['r_squared']:.3f}")
else:
    print("Insufficient data for power law fitting")

print(f"Energy dissipation rate: {cascade_analysis['dissipation_rate']:.6f}")
print(f"Integral length scale: {cascade_analysis['integral_scale']:.3f}")

## Advanced Visualization and Results

This section creates comprehensive visualizations of the fluid dynamics analysis, demonstrating how these theoretical concepts apply to financial market patterns.

In [ ]:
# Create comprehensive visualization
fig, axes = plt.subplots(3, 3, figsize=(18, 15))
fig.suptitle('Advanced Fluid Dynamics Analysis of Financial Markets', fontsize=16, fontweight='bold')

# 1. Velocity field with streamlines
x = np.arange(model.grid_size)
y = np.arange(model.grid_size)
X, Y = np.meshgrid(x[::5], y[::5])
U = model.velocity_x[::5, ::5]
V = model.velocity_y[::5, ::5]

axes[0,0].streamplot(X, Y, U, V, density=1.5, color='blue', alpha=0.6)
axes[0,0].set_title('Market Flow Field (Order Flow)')
axes[0,0].set_xlabel('Price Level')
axes[0,0].set_ylabel('Time')

# 2. Vorticity field
im1 = axes[0,1].imshow(vorticity_analysis['vorticity'], cmap='RdBu_r', origin='lower')
axes[0,1].set_title('Vorticity Field (Rotation Patterns)')
plt.colorbar(im1, ax=axes[0,1])

# 3. Reynolds number distribution
im2 = axes[0,2].imshow(regime_analysis['reynolds_field'], cmap='viridis', origin='lower')
axes[0,2].set_title(f'Reynolds Number Field\n(Regime: {regime_analysis["regime"]})')
plt.colorbar(im2, ax=axes[0,2])

# 4. Vortex strength classification
vortex_plot = np.zeros_like(vorticity_analysis['vorticity'])
vortex_plot[vortex_classification['weak_vortices']] = 1
vortex_plot[vortex_classification['moderate_vortices']] = 2
vortex_plot[vortex_classification['strong_vortices']] = 3

im3 = axes[1,0].imshow(vortex_plot, cmap='Reds', origin='lower')
axes[1,0].set_title('Vortex Strength Classification')
cbar = plt.colorbar(im3, ax=axes[1,0])
cbar.set_ticks([0, 1, 2, 3])
cbar.set_ticklabels(['None', 'Weak', 'Moderate', 'Strong'])

# 5. Energy spectrum
if cascade_analysis['slope'] is not None:
    axes[1,1].loglog(cascade_analysis['wavenumbers'], cascade_analysis['energy_spectrum'], 'bo-', alpha=0.7)
    
    # Plot theoretical Kolmogorov -5/3 line
    k_theory = cascade_analysis['wavenumbers']
    E_theory = cascade_analysis['amplitude'] * k_theory**(-5/3)
    axes[1,1].loglog(k_theory, E_theory, 'r--', label='Kolmogorov -5/3', alpha=0.8)
    
    axes[1,1].set_xlabel('Wavenumber k')
    axes[1,1].set_ylabel('Energy E(k)')
    axes[1,1].set_title(f'Energy Spectrum\n(Slope: {cascade_analysis["slope"]:.2f})')
    axes[1,1].legend()
    axes[1,1].grid(True, alpha=0.3)
else:
    axes[1,1].text(0.5, 0.5, 'Insufficient data\nfor spectrum analysis', 
                   ha='center', va='center', transform=axes[1,1].transAxes)
    axes[1,1].set_title('Energy Spectrum')

# 6. Turbulent kinetic energy
im4 = axes[1,2].imshow(regime_analysis['turbulent_kinetic_energy'], cmap='plasma', origin='lower')
axes[1,2].set_title('Turbulent Kinetic Energy')
plt.colorbar(im4, ax=axes[1,2])

# 7. Pressure field (synthetic)
pressure_field = -0.5 * (model.velocity_x**2 + model.velocity_y**2) + 0.1 * np.random.randn(*model.velocity_x.shape)
im5 = axes[2,0].imshow(pressure_field, cmap='coolwarm', origin='lower')
axes[2,0].set_title('Pressure Field (Market Pressure)')
plt.colorbar(im5, ax=axes[2,0])

# 8. Circulation strength
im6 = axes[2,1].imshow(vorticity_analysis['circulation_strength'], cmap='YlOrRd', origin='lower')
axes[2,1].set_title('Circulation Strength')
plt.colorbar(im6, ax=axes[2,1])

# 9. Regime summary statistics
axes[2,2].axis('off')
summary_text = f"""
MARKET REGIME ANALYSIS

Current Regime: {regime_analysis['regime'].upper()}
Confidence: {regime_analysis['confidence']:.1%}

Reynolds Number: {regime_analysis['mean_reynolds']:.1f}
Turbulence Intensity: {regime_analysis['turbulence_intensity']:.3f}
Regime Uniformity: {regime_analysis['uniformity']:.1%}

Energy Dissipation: {cascade_analysis['dissipation_rate']:.2e}
Total Circulation: {vorticity_analysis['total_circulation']:.2f}

Strong Vortices: {np.sum(vortex_classification['strong_vortices'])}
Moderate Vortices: {np.sum(vortex_classification['moderate_vortices'])}

TRADING IMPLICATIONS:
• {'Trend following recommended' if regime_analysis['regime'] == 'laminar' else 'Caution: volatile conditions' if regime_analysis['regime'] == 'turbulent' else 'Mixed signals - wait for clarity'}
• Vortex formations suggest {'high' if np.sum(vortex_classification['strong_vortices']) > 10 else 'moderate' if np.sum(vortex_classification['strong_vortices']) > 3 else 'low'} reversal probability
"""

axes[2,2].text(0.05, 0.95, summary_text, transform=axes[2,2].transAxes, 
               fontsize=10, verticalalignment='top', fontfamily='monospace',
               bbox=dict(boxstyle='round,pad=0.5', facecolor='lightgray', alpha=0.8))

plt.tight_layout()
plt.show()

print("\n" + "="*60)
print("ADVANCED FLUID DYNAMICS ANALYSIS COMPLETE")
print("="*60)
print(f"Market regime identified: {regime_analysis['regime'].upper()}")
print(f"Analysis confidence: {regime_analysis['confidence']:.1%}")
print(f"Key patterns detected: {np.sum(vortex_classification['strong_vortices'])} strong vortices")
print("\nThis analysis provides quantitative foundation for:")
print("• Regime detection and classification")
print("• Pattern recognition using fluid dynamics")
print("• Energy cascade analysis for market behavior")
print("• Advanced risk management strategies")
print("="*60)